# Import libraries

In [1]:
# built-in src
import os
import numpy as np
# third-party src
from keras import callbacks
import keras_tuner as kt
# project src
from src.data_aug import data_aug_with_random_contrast
from src.ranking_model_prob import create_meta_network

# Prepare data

In [2]:
# Configure question index
question_index = str(1)

# Load training data
training_data_path = r'D:\zluo033\DNNTraining\data\datasets\training_data\no_pref_removed'
img_left_training = np.load(os.path.join(training_data_path + '/train_left_duel_' + question_index + '.npy'), allow_pickle=True)
img_right_training = np.load(os.path.join(training_data_path + '/train_right_duel_' + question_index + '.npy'), allow_pickle=True)
label_training = np.load(os.path.join(training_data_path + '/train_label_duel_' + question_index + '.npy'), allow_pickle=True)

# Load validation data
validation_data_path = r'D:\zluo033\DNNTraining\data\datasets\validation_data\no_pref_removed'
img_left_validation = np.load(os.path.join(validation_data_path + '/val_left_duel_' + question_index + '.npy'), allow_pickle=True)
img_right_validation = np.load(os.path.join(validation_data_path + '/val_right_duel_' + question_index + '.npy'), allow_pickle=True)
label_validation = np.load(os.path.join(validation_data_path + '/val_label_duel_' + question_index + '.npy'), allow_pickle=True)

# Load test data
test_data_path = r'D:\zluo033\DNNTraining\data\datasets\test_data\no_pref_removed'
img_left_test = np.load(os.path.join(test_data_path + '/test_left_duel_' + question_index + '.npy'), allow_pickle=True)
img_right_test = np.load(os.path.join(test_data_path + '/test_right_duel_' + question_index + '.npy'), allow_pickle=True)
label_test = np.load(os.path.join(test_data_path + '/test_label_duel_' + question_index + '.npy'), allow_pickle=True)

## Augment data with random contrast
*This is a data augmentation method that raised by Hannick. It helps to improve the model performance.*
[Hannick's GitHub](https://github.com/Hannick5/UrbanPerception)

In [3]:
# Data augmentation using random contrast
img_left_training,img_right_training,label_training = data_aug_with_random_contrast(img_left_training,img_right_training, label_training,save_folder=None)

In [4]:
# Check data
print(img_left_training.shape,img_left_validation.shape,label_training.shape)

(5474, 224, 224, 3) (913, 224, 224, 3) (5474,)


# Configure a hypermodel tuned

In [5]:
# Construct a hypermodel by defining the search space of selected hyperparameters
def build_ranking_hypermodel(hp):
    # Define model architecture
    model = create_meta_network(
        dense_units=hp.Choice('dense_units', values=[32, 64, 128, 256, 512]),
        dropout_rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.6, step=0.2),
        learning_rate=hp.Choice('learning_rate', values=[1e-4, 1e-5, 1e-6]),
        optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
        activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid']),
        learning_rate_decay=hp.Choice('learning_rate_decay', values=[1e-3, 1e-4, 1e-5]),
        img_size=int(224),
        data_aug=True,
        weights=None
    )
    return model
build_ranking_hypermodel(kt.HyperParameters())

# Instantiate a tuner adopting HyperBand approach
search_result_dir = 'HyperModels'
project_name = 'hyperband_search_trials'
tuner = kt.Hyperband(
    build_ranking_hypermodel,
    objective='val_accuracy',  # The metric to optimize during hyperparameter search
    max_epochs=100,             # Maximum number of epochs to train each model
    factor=3,                  # Reduction factor for the number of epochs and resource allocation
    directory=search_result_dir,        # Directory to store the search results
    project_name=project_name  # Name for the project (a folder will be created inside 'search_result_dir')
)

# Employ early stopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode='min',
    restore_best_weights=True
)

INFO:tensorflow:Reloading Tuner from D:\zluo033\DNNTraining\models\Q1\VGGNet19\HyperModels\hyperband_search\tuner0.json


# Search the best combination of hyperparameters tuned

In [6]:
# Tune the hypermodel
tuner.search(
    x=[img_left_training, img_right_training],
    y=label_training,
    epochs=50,
    batch_size=32,
    validation_data=([img_left_validation, img_right_validation], label_validation),
    callbacks=[early_stopping]
)

tuner.search(
    x=[img_left_training,img_right_training],  # Input features for training
    y=label_training,                          # Target labels for training
    validation_data=([img_left_validation,img_right_validation], label_validation),   # Validation data for early stopping and performance evaluation
    callbacks=[early_stopping],         # Early stopping callback
    verbose=1,                          # Verbosity level (0: silent, 1: progress bar, 2: one line per epoch)
    batch_size=32,                      # Batch size for training
)

Trial 165 Complete [00h 01m 14s]
val_accuracy: 0.8532311320304871

Best val_accuracy So Far: 0.8532311320304871
Total elapsed time: 02h 19m 00s

Search: Running Trial #166

Value             |Best Value So Far |Hyperparameter
256               |256               |dense_units
0                 |0                 |dropout_rate
1e-06             |0.0001            |learning_rate
sgd               |adam              |optimizer
sigmoid           |relu              |activation
1e-05             |1e-05             |learning_rate_decay
4                 |4                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/4
172/172 [==============================] - 18s 99ms/step - loss: 1.0073 - accuracy: 0.5163 - val_loss: 0.8293 - val_accuracy: 0.5279
Epoch 2/4
172/172 [==============================] - 16s 93ms/step - loss: 0.9840 - accuracy: 0.5230 - val_loss

# Obtain the best hypermodel

In [9]:
# Display the tuning process summary
tuner.results_summary()

In [10]:
# Save the best five hypermodels
best_hps = tuner.get_best_hyperparameters(5)
save_dir = 'HyperModels/ranking_hypermodels/hyperband_search_results'
for i in range(len(best_hps)):
    model = build_ranking_hypermodel(best_hps[i])
    model.save(os.path.join(save_dir, 'best_hypermodel_' + str(i+1) + '.h5'))

# Retrain the best hypermodel

In [11]:
# Retrain the best hypermodel
best_hypermodel = tuner.get_best_models(1)[0]
best_hypermodel.summary()

best_hypermodel.compile(
    optimizer=best_hypermodel.optimizer,
    loss=best_hypermodel.loss,
    metrics=['accuracy']
)

best_hypermodel_history = best_hypermodel.fit(
    x=[img_left_training, img_right_training],
    y=label_training,
    epochs=50,
    batch_size=32,
    validation_data=([img_left_validation, img_right_validation], label_validation),
    callbacks=[early_stopping]
)

# Plot the training history
import matplotlib.pyplot as plt
plt.plot(best_hypermodel_history.history['accuracy'])
plt.plot(best_hypermodel_history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')